In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import (MinMaxInterval, LogStretch,
                                 ImageNormalize, simple_norm)
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground
from photutils.detection import detect_sources, deblend_sources
from photutils.segmentation import SourceCatalog
from photutils.aperture import CircularAperture
from astroquery.mast import Observations

# JWST Data Analysis: SMACS 0723

This notebook demonstrates how to:
1. Query JWST NIRCam imaging data using astroquery
2. Download and display calibrated images
3. Apply contrast stretching
4. Overlay celestial coordinates and scale bar
5. Extract source positions

We'll be following PEP-8 guidelines and using Python 3.12 for this analysis.

# Import Required Libraries

We'll import all the necessary libraries for our analysis:

In [4]:
# Standard imports
import os
import numpy as np
from matplotlib import pyplot as plt

# Astropy imports
from astropy.io import fits
from astropy.visualization import (LogStretch, MinMaxInterval,
                                ImageNormalize, simple_norm)
from astropy.wcs import WCS
import astropy.units as u

# Astroquery imports
from astroquery.mast import Observations

# Photutils imports
from photutils.detection import DAOStarFinder
from photutils.background import Background2D, MedianBackground

# Set up matplotlib for notebook display
%matplotlib inline
#plt.style.use('seaborn-darkgrid')

# Search for JWST NIRCam Imaging Data

We'll search for public JWST NIRCam imaging data for SMACS 0723 using the MAST archive:

In [5]:
# Define search parameters
target = "SMACS-0723"
instrument = "NIRCAM"
filt = "F200W"

# Search for observations
obs = Observations.query_criteria(
    target_name=target,
    instrument_name=instrument,
    filters=filt,
    obs_collection="JWST"
)

print(f"Found {len(obs)} observations matching our criteria.")
if len(obs) > 0:
    print("\nFirst observation details:")
    print(obs[0])

Found 0 observations matching our criteria.


# Download and Display Calibrated Image

Now we'll download a calibrated image from the F200W filter and display it:

In [6]:
# Get the data products
products = Observations.get_product_list(obs)
calibrated = products[products['productSubGroupDescription'] == 'CAL']
f200w = calibrated[calibrated['filters'] == 'F200W']

# Download the data
if len(f200w) > 0:
    download = Observations.download_products(f200w[0:1])
    filename = download['Local Path'][0]
    
    # Read the FITS file
    with fits.open(filename) as hdul:
        image_data = hdul[1].data
        header = hdul[1].header
        wcs = WCS(header)
    
    # Create a simple display of the image
    plt.figure(figsize=(12, 10))
    plt.imshow(image_data, cmap='gray')
    plt.colorbar(label='Flux')
    plt.title('JWST NIRCam F200W Image of SMACS 0723')
    plt.show()
else:
    print("No F200W calibrated data found.")

InvalidQueryError: Observation list is empty, no associated products.

# Apply Contrast Stretching

We'll apply contrast stretching using LogStretch and MinMaxInterval from astropy.visualization:

In [ ]:
# Create the normalization using LogStretch and MinMaxInterval
norm = ImageNormalize(
    image_data,
    interval=MinMaxInterval(),
    stretch=LogStretch()
)

# Display the contrast-stretched image
plt.figure(figsize=(12, 10))
plt.imshow(image_data, norm=norm, cmap='gray')
plt.colorbar(label='Flux (log scale)')
plt.title('JWST NIRCam F200W Image (Log Stretched)')
plt.show()

# Overlay Celestial Coordinates and Scale Bar

Now we'll create a plot with WCS coordinates and add a scale bar:

In [ ]:
# Create a figure with WCS projection
plt.clf()  # Clear any existing plots
fig = plt.figure(figsize=(12, 10))

# Create WCS axes
ax = plt.subplot(projection=wcs)

# Display the image with proper normalization
im = ax.imshow(image_data, norm=norm, origin='lower', cmap='gray')

# Configure the coordinate grid
ax.coords.grid(True, color='white', ls='solid', alpha=0.3)
ax.coords[0].set_axislabel('Right Ascension (J2000)')
ax.coords[1].set_axislabel('Declination (J2000)')

# Add a colorbar
plt.colorbar(im, label='Flux (log scale)', ax=ax)

# Add title
plt.title('JWST NIRCam F200W Image with Celestial Coordinates')

# Add a scale bar (assuming the WCS information is correct)
# Convert 1 arcsec to degrees for the scale bar
scale_bar_length = 1/3600  # 1 arcsec in degrees
add_scalebar(ax, scale_bar_length, label='1 arcsec', color='white',
            corner='bottom right', frame=True)

plt.tight_layout()
plt.show()

# Extract Source Positions

Finally, we'll use photutils to detect and extract source positions:

In [ ]:
# Estimate the background
sigma_clip = SigmaClip(sigma=3.0)
bkg_estimator = MedianBackground()
bkg = Background2D(image_data, (50, 50), filter_size=(3, 3),
                  sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

# Detect sources
threshold = 3. * bkg.background_rms
sources = detect_sources(image_data - bkg.background, threshold, npixels=5)

# Deblend sources
deblended = deblend_sources(image_data - bkg.background, sources, npixels=5,
                           filter_kernel=None, nlevels=32, contrast=0.001)

# Calculate source properties
catalog = SourceCatalog(image_data - bkg.background, deblended, wcs=wcs)
tbl = catalog.to_table()
print("\nDetected Sources:")
print(tbl['label', 'xcentroid', 'ycentroid', 'sky_centroid'])

# Plot the image with detected sources
plt.clf()
fig = plt.figure(figsize=(12, 10))
ax = plt.subplot(projection=wcs)

# Display the image
im = ax.imshow(image_data, norm=norm, origin='lower', cmap='gray')

# Plot source positions with apertures
positions = np.transpose((tbl['xcentroid'], tbl['ycentroid']))
apertures = CircularAperture(positions, r=5.)
apertures.plot(color='red', lw=1.5, alpha=0.7)

# Configure the coordinate grid
ax.coords.grid(True, color='white', ls='solid', alpha=0.3)
ax.coords[0].set_axislabel('Right Ascension (J2000)')
ax.coords[1].set_axislabel('Declination (J2000)')

# Add a colorbar
plt.colorbar(im, label='Flux (log scale)', ax=ax)

# Add title
plt.title('Detected Sources in JWST NIRCam F200W Image')

# Add a scale bar
scale_bar_length = 1/3600  # 1 arcsec in degrees
add_scalebar(ax, scale_bar_length, label='1 arcsec', color='white',
            corner='bottom right', frame=True)

plt.tight_layout()
plt.show()

# Conclusion

We have successfully:
1. Retrieved JWST NIRCam F200W data for SMACS 0723
2. Applied appropriate contrast stretching
3. Created a visualization with celestial coordinates and a scale bar
4. Detected and extracted source positions

The detected sources can be used for further analysis such as photometry or astrometry.